In [1]:
import os
import numpy as np
import pandas as pd
import gzip

In [2]:
os.chdir('E:/CNSI_project/demo_and_data_3.0/Raw_data/eICU/')
pd.set_option('display.max_columns', None)

In [3]:
filename = './microLab.csv.gz'
with gzip.open(filename, 'rt', encoding='utf-8') as file:
    df = pd.read_csv(file)
df

,microlabid,patientunitstayid,culturetakenoffset,culturesite,organism,antibiotic,sensitivitylevel
0,549559,983524,-688,"Blood, Venipuncture",gram positive rods,NaN,NaN
1,549555,983524,-5481,Stool,no growth,NaN,NaN
2,549565,983524,324,Stool,no growth,NaN,NaN
3,549562,983524,-686,"Blood, Central Line",gram negative rods,NaN,NaN
4,549564,983524,-681,"Blood, Central Line",gram positive cocci,NaN,NaN
...,...,...,...,...,...,...,...
16991,1617032,3160582,411,Other,mixed flora,NaN,NaN
16992,1617031,3160582,-521,"Blood, Venipuncture",no growth,NaN,NaN
16993,1617033,3160582,411,Other,Candida albicans,NaN,NaN
16994,1617030,3160582,-522,"Blood, Venipuncture",no growth,NaN,NaN


In [4]:
print([each for each in df['culturesite'].unique() if 'CSF' in each])
df['culturesite'].unique()

['CSF']


array(['Blood, Venipuncture', 'Stool', 'Blood, Central Line',
       'Urine, Voided Specimen', 'Sputum, Tracheal Specimen',
       'Nasopharynx', 'Urine, Catheter Specimen', 'Sputum, Expectorated',
       'Bronchial Lavage', 'CSF', 'Other', 'Wound, Drainage Fluid',
       'Wound, Decubitus', 'Abscess', 'Pleural Fluid', 'Peritoneal Fluid',
       'Wound, Surgical', 'Bile', 'Rectal Swab', 'Skin'], dtype=object)

In [6]:
# 提取出做了CSF菌培养的病人
df1 = df[df['culturesite'].str.contains('CSF')]
df1

,microlabid,patientunitstayid,culturetakenoffset,culturesite,organism,antibiotic,sensitivitylevel
83,551846,1047047,15008,CSF,no growth,NaN,NaN
93,551984,1047047,17408,CSF,no growth,NaN,NaN
97,550659,1047047,3043,CSF,no growth,NaN,NaN
161,834790,2573296,-130,CSF,no growth,NaN,NaN
343,837305,2576788,1230,CSF,no growth,NaN,NaN
...,...,...,...,...,...,...,...
16121,1627693,3157971,900,CSF,no growth,NaN,NaN
16141,1634331,3158058,8389,CSF,no growth,NaN,NaN
16143,1634332,3158058,8389,CSF,no growth,NaN,NaN
16462,1610437,3158913,-127,CSF,no growth,NaN,NaN


In [8]:
df1[['organism']].value_counts()

organism                       
no growth                          57
Other                              31
Streptococcus pneumoniae            4
yeast                               2
Staphylococcus aureus               1
gram positive cocci                 1
gram positive cocci - in chains     1
dtype: int64

In [10]:
# other被我判为阴性，不知道是否妥当
list1 = ['no growth','Other']
list2 = ['Streptococcus pneumoniae','yeast','Staphylococcus aureus','gram positive cocci','gram positive cocci - in chains']
def culture(outcome):
    if outcome in list1:
        return 0
    elif outcome in list2:
        return 1
    else:
        np.nan
df2 = df1.copy()
df2['culture'] = df2['organism'].apply(lambda x: culture(x))
df2

,microlabid,patientunitstayid,culturetakenoffset,culturesite,organism,antibiotic,sensitivitylevel,culture
83,551846,1047047,15008,CSF,no growth,NaN,NaN,0
93,551984,1047047,17408,CSF,no growth,NaN,NaN,0
97,550659,1047047,3043,CSF,no growth,NaN,NaN,0
161,834790,2573296,-130,CSF,no growth,NaN,NaN,0
343,837305,2576788,1230,CSF,no growth,NaN,NaN,0
...,...,...,...,...,...,...,...,...
16121,1627693,3157971,900,CSF,no growth,NaN,NaN,0
16141,1634331,3158058,8389,CSF,no growth,NaN,NaN,0
16143,1634332,3158058,8389,CSF,no growth,NaN,NaN,0
16462,1610437,3158913,-127,CSF,no growth,NaN,NaN,0


In [11]:
df2['culture'].value_counts()

0    88
1     9
Name: culture, dtype: int64

In [15]:
# 挑出同一次住院产生不同菌培养结果的情况
groups = df2.groupby(['patientunitstayid'])['culture'].apply(lambda x: x.dropna().nunique()>1)
filtered_data = groups[groups].reset_index()[['patientunitstayid']]
df_filtered = df2.merge(filtered_data, on=['patientunitstayid'],how='inner')
df_filtered

,microlabid,patientunitstayid,culturetakenoffset,culturesite,organism,antibiotic,sensitivitylevel,culture
0,839204,2583429,-2858,CSF,no growth,NaN,NaN,0
1,839108,2583429,-12995,CSF,Other,NaN,NaN,0
2,840020,2583429,-12995,CSF,yeast,NaN,NaN,1


In [18]:
(12995-2858)/60/24

7.039583333333333

In [29]:
# 若同一个病人多次采样的结果不同，且不同结果的采样时间相差在24小时以内，则将他们的culture结果都变为1
df3 = df2.copy()
df3.sort_values(['patientunitstayid', 'culturetakenoffset'], inplace=True)

def adjust_culture(group):
    # 如果病人只有一次采样，或者所有采样结果都相同，不需要修改
    if len(group['culture'].unique()) == 1:
        return group
    else:
        # 找出时间差在24小时以内的连续采样
        group['time_diff'] = group['culturetakenoffset'].diff()
        mask = (group['culture'].diff().abs() == 1) & (group['time_diff'] <= 1440)
        # 如果存在这样的采样，将这两次culture结果变为1
        group.loc[mask | mask.shift(-1), 'culture'] = 1
        return group

df3 = df3.groupby('patientunitstayid').apply(adjust_culture)

# 删除创建的time_diff列
df3.drop(columns='time_diff', inplace=True)
df3

,microlabid,patientunitstayid,culturetakenoffset,culturesite,organism,antibiotic,sensitivitylevel,culture
97,550659,1047047,3043,CSF,no growth,NaN,NaN,0
83,551846,1047047,15008,CSF,no growth,NaN,NaN,0
93,551984,1047047,17408,CSF,no growth,NaN,NaN,0
161,834790,2573296,-130,CSF,no growth,NaN,NaN,0
343,837305,2576788,1230,CSF,no growth,NaN,NaN,0
...,...,...,...,...,...,...,...,...
16121,1627693,3157971,900,CSF,no growth,NaN,NaN,0
16141,1634331,3158058,8389,CSF,no growth,NaN,NaN,0
16143,1634332,3158058,8389,CSF,no growth,NaN,NaN,0
16462,1610437,3158913,-127,CSF,no growth,NaN,NaN,0


In [30]:
df3['culture'].value_counts()

0    87
1    10
Name: culture, dtype: int64

In [31]:
df3[df3['patientunitstayid']==2583429]

,microlabid,patientunitstayid,culturetakenoffset,culturesite,organism,antibiotic,sensitivitylevel,culture
569,839108,2583429,-12995,CSF,Other,NaN,NaN,1
572,840020,2583429,-12995,CSF,yeast,NaN,NaN,1
568,839204,2583429,-2858,CSF,no growth,NaN,NaN,0


In [44]:
# 若同一个病人在24小时内有多次相同的采样结果，则只取第一个结果的行数据。
df3.sort_values(['patientunitstayid', 'culturetakenoffset'], inplace=True)

def remove_duplicates(group):
    # 如果病人只有一次采样，不需要修改
    if len(group) == 1:
        return group
    else:
        # 找出在24小时内有多次相同的采样结果
        group['time_diff'] = group['culturetakenoffset'].diff()
        mask = (group['culture'] == group['culture'].shift()) & (group['time_diff'] <= 1440)
        # 删除这些重复的结果，只保留第一个
        group = group[~mask]
        return group

df3 = df3.groupby('patientunitstayid').apply(remove_duplicates)

# 删除创建的time_diff列
df3.drop(columns='time_diff', inplace=True)
df3

microlabid  patientunitstayid  culturetakenoffset  \
patientunitstayid                                                            
1047047           97         550659            1047047                3043   
                  83         551846            1047047               15008   
                  93         551984            1047047               17408   
2573296           161        834790            2573296                -130   
2576788           343        837305            2576788                1230   
...                             ...                ...                 ...   
3155904           15473     1615828            3155904                5694   
3157971           16121     1627693            3157971                 900   
3158058           16141     1634331            3158058                8389   
3158913           16462     1610437            3158913                -127   
3159526           16655     1625453            3159526                8816   

                        culturesite   organism antibiotic sensitivitylevel  \
patientunitstayid                                                            
1047047           97            CSF  no growth        NaN              NaN   
                  83            CSF  no growth        NaN              NaN   
                  93            CSF  no growth        NaN              NaN   
2573296           161           CSF  no growth        NaN              NaN   
2576788           343           CSF  no growth        NaN              NaN   
...                             ...        ...        ...              ...   
3155904           15473         CSF  no growth        NaN              NaN   
3157971           16121         CSF  no growth        NaN              NaN   
3158058           16141         CSF  no growth        NaN              NaN   
3158913           16462         CSF  no growth        NaN              NaN   
3159526           16655         CSF  no growth        NaN              NaN   

                         culture  
patientunitstayid                 
1047047           97           0  
                  83           0  
                  93           0  
2573296           161          0  
2576788           343          0  
...                          ...  
3155904           15473        0  
3157971           16121        0  
3158058           16141        0  
3158913           16462        0  
3159526           16655        0  

[67 rows x 8 columns]

In [45]:
df3['culture'].value_counts()

0    63
1     4
Name: culture, dtype: int64

In [46]:
df3[df3['patientunitstayid']==2583429]

microlabid  patientunitstayid  culturetakenoffset  \
patientunitstayid                                                          
2583429           569      839108            2583429              -12995   
                  568      839204            2583429               -2858   

                      culturesite   organism antibiotic sensitivitylevel  \
patientunitstayid                                                          
2583429           569         CSF      Other        NaN              NaN   
                  568         CSF  no growth        NaN              NaN   

                       culture  
patientunitstayid               
2583429           569        1  
                  568        0

In [60]:
df3 = df3.reset_index(drop=True)
df3

,microlabid,patientunitstayid,culturetakenoffset,culturesite,organism,antibiotic,sensitivitylevel,culture
0,550659,1047047,3043,CSF,no growth,NaN,NaN,0
1,551846,1047047,15008,CSF,no growth,NaN,NaN,0
2,551984,1047047,17408,CSF,no growth,NaN,NaN,0
3,834790,2573296,-130,CSF,no growth,NaN,NaN,0
4,837305,2576788,1230,CSF,no growth,NaN,NaN,0
...,...,...,...,...,...,...,...,...
62,1615828,3155904,5694,CSF,no growth,NaN,NaN,0
63,1627693,3157971,900,CSF,no growth,NaN,NaN,0
64,1634331,3158058,8389,CSF,no growth,NaN,NaN,0
65,1610437,3158913,-127,CSF,no growth,NaN,NaN,0


In [64]:
df3['patientunitstayid'].value_counts()

1047047    3
2602968    3
2583429    2
3104980    2
2644964    2
2582617    1
3139600    1
3077633    1
2576788    1
3123397    1
3131170    1
3131763    1
3131909    1
3133623    1
3135561    1
3136367    1
3137362    1
3139077    1
3141147    1
3055844    1
3141371    1
3142740    1
3144241    1
3144382    1
3147081    1
3153395    1
3155519    1
3155904    1
3157971    1
3158058    1
3158913    1
3064546    1
3055799    1
2582970    1
2602402    1
2583126    1
2581002    1
2583789    1
2588350    1
2588558    1
2589249    1
2590832    1
2597361    1
2597831    1
2601479    1
2601903    1
2602626    1
3035720    1
2578536    1
2604537    1
2605795    1
2608009    1
2615832    1
2618787    1
2626224    1
2638728    1
2638853    1
2640095    1
2573296    1
3159526    1
Name: patientunitstayid, dtype: int64

# 提取以及合并lab的数据

In [32]:
filename = './lab.csv.gz'
with gzip.open(filename, 'rt', encoding='utf-8') as file:
    df4 = pd.read_csv(file)
df4

,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset
0,52307161,141168,2026,3,fibrinogen,177.00,177,mg/dL,mg/dL,2219
1,50363251,141168,1133,3,PT - INR,2.50,2.5,ratio,NaN,1208
2,49149139,141168,2026,1,magnesium,2.00,2.0,mg/dL,mg/dL,2090
3,50363250,141168,1133,3,PT,26.60,26.6,sec,sec,1208
4,66695374,141168,2141,7,pH,7.20,7.20,NaN,Units,2155
...,...,...,...,...,...,...,...,...,...,...
39132526,824772675,3353263,-7,3,WBC x 1000,6.40,6.4,K/mcL,K/uL,6
39132527,826470517,3353263,1733,3,RBC,4.67,4.67,M/mcL,M/uL,1774
39132528,824772678,3353263,-7,3,-monos,10.00,10,%,%,6
39132529,826470516,3353263,1733,3,WBC x 1000,6.60,6.6,K/mcL,K/uL,1774


In [49]:
# 从lab数据中选出做过CSF菌培养的patientunitstayid
df5 = df4.copy()
df5 = df5[df5['patientunitstayid'].isin(df3['patientunitstayid'])]
df5

,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset
11131632,245475050,1047047,32548,3,PT - INR,1.000,1.0,ratio,NaN,32587
11131633,247443294,1047047,29399,4,bedside glucose,128.000,128,mg/dL,mg/dL,29401
11131634,248022362,1047047,8135,4,urinary specific gravity,1.022,1.022,NaN,NaN,8152
11131635,247202351,1047047,27619,4,bedside glucose,95.000,95,mg/dL,mg/dL,27620
11131636,248351985,1047047,38353,4,bedside glucose,125.000,125,mg/dL,mg/dL,38354
...,...,...,...,...,...,...,...,...,...,...
35917083,774220676,3159526,28240,1,total bilirubin,0.200,0.2,mg/dL,mg/dL,28277
35917084,773979508,3159526,15267,1,sodium,141.000,141,mmol/L,mmol/L,15321
35917085,774236801,3159526,10985,1,ALT (SGPT),150.000,150,Units/L,U/L,11064
35917086,774054328,3159526,28240,1,phosphate,5.500,5.5,mg/dL,mg/dL,28277


In [50]:
# 根据MIMIC中的feature从eICU中挑选出有用的feature
df6 = df5.copy()
useful_labname = ['lactate','-lymphs','-eos','RBC','WBC x 1000','-polys', '-monos','-basos','glucose', 'total protein', '-bands',"WBC's in cerebrospinal fluid",'protein - CSF', 'glucose - CSF']
df6 = df6[df6['labname'].isin(useful_labname)]
df6

,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset
11131647,247023943,1047047,15008,4,WBC's in cerebrospinal fluid,142.00,142,NaN,/cumm,15074
11131657,246741683,1047047,10113,3,RBC,3.31,3.31,M/mcL,/pL,10127
11131662,243816843,1047047,4443,3,RBC,3.44,3.44,M/mcL,/pL,4468
11131664,242636657,1047047,14532,3,RBC,3.12,3.12,M/mcL,/pL,14569
11131671,247283772,1047047,15008,4,glucose - CSF,72.00,72,mg/dL,mg/dL,15065
...,...,...,...,...,...,...,...,...,...,...
35917049,773776855,3159526,8097,1,total protein,5.00,5.0,g/dL,gm/dL,8274
35917065,773859442,3159526,13881,1,total protein,5.60,5.6,g/dL,gm/dL,13921
35917070,774236802,3159526,10985,1,glucose,135.00,135,mg/dL,mg/dL,11064
35917077,774047240,3159526,22507,1,glucose,100.00,100,mg/dL,mg/dL,22563


In [73]:
df6['labname'].unique().shape

(14,)

In [94]:
# 合并df3和df6
df = pd.merge(df3, df6, on='patientunitstayid')

# 计算时间差，筛选出在24小时内的检测结果
df['time_diff'] = (df['labresultoffset'] - df['culturetakenoffset']).abs()
df = df[df['time_diff'] <= 60*24*1]

# 删除我们创建的time_diff列
df.drop(columns='time_diff', inplace=True)

# 将labresult列转为字符串
df['labresult'] = df['labresult'].astype(str)

# 将每种检测结果转为一列，并用~连接多次检测的结果
df_result = df.groupby(['patientunitstayid', 'culturetakenoffset', 'culture', 'labname'])['labresult'].apply('~'.join).unstack()

# 重置index
df_result.reset_index(inplace=True)
df_result.columns.name = None  # 摆脱dataframe python中显示在索引顶部/上方的列/轴名称：重命名轴
df_result

labname,patientunitstayid,culturetakenoffset,culture,-bands,-basos,-eos,-lymphs,-monos,-polys,RBC,WBC x 1000,WBC's in cerebrospinal fluid,glucose,glucose - CSF,lactate,protein - CSF,total protein
0,1047047,3043,0,NaN,NaN,NaN,NaN,NaN,NaN,3.44~3.21,14.6~13.5,112.0,129.0~106.0,69.0,NaN,177.0,NaN
1,1047047,15008,0,NaN,NaN,NaN,NaN,NaN,NaN,3.12~3.04,17.1~16.0,142.0,107.0~118.0,72.0,NaN,69.0,NaN
2,1047047,17408,0,NaN,NaN,NaN,NaN,NaN,NaN,2.96,17.2,NaN,102.0,NaN,NaN,NaN,NaN
3,2573296,-130,0,NaN,NaN,0.0,7.0,5.0,88.0,5.15,12.55,108.0,93.0~102.0,51.0,NaN,NaN,8.4
4,2576788,1230,0,NaN,NaN,0.0~0.0,4.0~7.0,12.0~8.0,87.0~79.0,4.42~3.9,17.61~14.38,1.0~7.0,112.0~101.0~135.0,74.0,1.3,54.0,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,3155904,5694,0,NaN,0.5~0.9,0.0~0.0,9.3~4.0,3.4~1.5,83.4~93.0,2.81~2.76,9.5~9.7,0.0,124.0~153.0,90.0,4.4~6.4,19.0,5.2
59,3157971,900,0,NaN,1.3~1.4~1.0,0.4~1.1~0.2,18.8~17.3~18.6,3.2~7.3~3.6,73.6~75.6~76.6,5.07~5.47~4.66,9.3~12.4~9.6,8.0,163.0~347.0~211.0~117.0,109.0,3.5~2.2,65.0,4.8~8.4~6.8
60,3158058,8389,0,2.0~3.0,0.5,2.0,6.0~13.0,9.0~6.0,83.0~72.0,3.49~3.53,12.9~9.5,1.0,116.0~108.0,65.0,NaN,202.0,NaN
61,3158913,-127,0,3.5,0.2,0.0,21.0,6.5,69.0,3.31~4.33,36.3~20.6,0.0,141.0~214.0~172.0,109.0,12.8~1.4,50.3,5.8


In [104]:
# 如果一项检测有多个数据，则默认选最后一个数据
# 定义一个函数，用于将多个数字用~连接在一起的字符串转换为数值型并取最后一个值
def str_to_last(x):
    if isinstance(x, str) and '~' in x:
        nums = [float(num) for num in x.split('~')]
        return nums[-1]
    elif isinstance(x, str):
        return float(x)
    else:
        return x

# 对第三列及以后的列应用上述函数
df_result.iloc[:, 3:] = df_result.iloc[:, 3:].applymap(str_to_last)

df_result

,patientunitstayid,culturetakenoffset,culture,-bands,-basos,-eos,-lymphs,-monos,-polys,RBC,WBC x 1000,WBC's in cerebrospinal fluid,glucose,glucose - CSF,lactate,protein - CSF,total protein
0,1047047,3043,0,NaN,NaN,NaN,NaN,NaN,NaN,3.21,13.50,112.0,106.0,69.0,NaN,177.0,NaN
1,1047047,15008,0,NaN,NaN,NaN,NaN,NaN,NaN,3.04,16.00,142.0,118.0,72.0,NaN,69.0,NaN
2,1047047,17408,0,NaN,NaN,NaN,NaN,NaN,NaN,2.96,17.20,NaN,102.0,NaN,NaN,NaN,NaN
3,2573296,-130,0,NaN,NaN,0.0,7.0,5.0,88.0,5.15,12.55,108.0,102.0,51.0,NaN,NaN,8.4
4,2576788,1230,0,NaN,NaN,0.0,7.0,8.0,79.0,3.90,14.38,7.0,135.0,74.0,1.3,54.0,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,3155904,5694,0,NaN,0.9,0.0,4.0,1.5,93.0,2.76,9.70,0.0,153.0,90.0,6.4,19.0,5.2
59,3157971,900,0,NaN,1.0,0.2,18.6,3.6,76.6,4.66,9.60,8.0,117.0,109.0,2.2,65.0,6.8
60,3158058,8389,0,3.0,0.5,2.0,13.0,6.0,72.0,3.53,9.50,1.0,108.0,65.0,NaN,202.0,NaN
61,3158913,-127,0,3.5,0.2,0.0,21.0,6.5,69.0,4.33,20.60,0.0,172.0,109.0,1.4,50.3,5.8


In [105]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   patientunitstayid             63 non-null     int64  
 1   culturetakenoffset            63 non-null     int64  
 2   culture                       63 non-null     int64  
 3   -bands                        12 non-null     float64
 4   -basos                        21 non-null     float64
 5   -eos                          38 non-null     float64
 6   -lymphs                       45 non-null     float64
 7   -monos                        45 non-null     float64
 8   -polys                        38 non-null     float64
 9   RBC                           62 non-null     float64
 10  WBC x 1000                    62 non-null     float64
 11  WBC's in cerebrospinal fluid  40 non-null     float64
 12  glucose                       62 non-null     float64
 13  glucose